In [0]:
!pip install larq

In [0]:
import tensorflow as tf
import larq as lq
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np

# learning rate tuning during learning

def lr_schedule(epoch):
    lrate = 0.01
    if epoch > 25:
        lrate = 0.001
    if epoch > 50:
        lrate = 0.0005        
    return lrate


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

weight_decay = 1e-4


kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip"
              )

model = tf.keras.models.Sequential([
    # In the first layer we only quantize the weights and not the input
    lq.layers.QuantConv2D(32, (3,3),
                          padding="same",
                          kernel_quantizer="ste_sign",
                          kernel_constraint="weight_clip",
                          activation='relu',
                          input_shape=x_train.shape[1:]),
    #tf.keras.layers.Activation('relu'),
    tf.keras.layers.BatchNormalization(),
    lq.layers.QuantConv2D(32, (3,3), padding="same", activation='relu',**kwargs),
    #tf.keras.layers.Activation('relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    lq.layers.QuantConv2D(64, (3,3), padding="same", activation='relu',**kwargs),
    #tf.keras.layers.Activation('relu'),
    tf.keras.layers.BatchNormalization(),
    lq.layers.QuantConv2D(64, (3,3), padding="same", activation='relu',**kwargs),
    #tf.keras.layers.Activation('relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    lq.layers.QuantConv2D(128, (3,3), padding="same", activation='relu',**kwargs),
    #tf.keras.layers.Activation('relu'),
    tf.keras.layers.BatchNormalization(),
    lq.layers.QuantConv2D(128, (3,3), padding="same", activation='relu',**kwargs),
    #tf.keras.layers.Activation('relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    lq.layers.QuantDense(10, **kwargs,activation='softmax')
])




datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

# training
batch_size = 64

opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

lq.models.summary(model)

+sequential_10 stats----------------------------------------------------------------------------------------+
| Layer                   Input prec.           Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs  32-bit MACs |
|                               (bit)                        x 1       x 1    (kB)                          |
+-----------------------------------------------------------------------------------------------------------+
| quant_conv2d_60                   -  (-1, 32, 32, 32)      864        32    0.23           0       884736 |
| batch_normalization_66            -  (-1, 32, 32, 32)        0        64    0.25           0            0 |
| quant_conv2d_61                   1  (-1, 32, 32, 32)     9216        32    1.25     9437184            0 |
| batch_normalization_67            -  (-1, 32, 32, 32)        0        64    0.25           0            0 |
| max_pooling2d_30                  -  (-1, 16, 16, 32)        0         0       0           0            0 |
| quant_co

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=75,
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/75
781/781 [==============================] - 47s 60ms/step - loss: 138.2786 - accuracy: 0.1950 - val_loss: 72.7854 - val_accuracy: 0.2713 - lr: 0.0100
Epoch 2/75
781/781 [==============================] - 47s 60ms/step - loss: 90.6262 - accuracy: 0.2574 - val_loss: 96.1414 - val_accuracy: 0.3090 - lr: 0.0100
Epoch 3/75
781/781 [==============================] - 46s 59ms/step - loss: 71.7008 - accuracy: 0.3015 - val_loss: 60.0726 - val_accuracy: 0.2988 - lr: 0.0100
Epoch 4/75
781/781 [==============================] - 47s 60ms/step - loss: 60.5547 - accuracy: 0.3223 - val_loss: 66.1529 - val_accuracy: 0.2686 - lr: 0.0100
Epoch 5/75
781/781 [==============================] - 47s 60ms/step - loss: 50.6748 - accuracy: 0.3446 - val_loss: 54.0107 - val_accuracy: 0.3230 - lr: 0.0100
Epoch 6/75
781/781 [==============================] - 46s 60ms/step - loss: 44.5796 - accuracy: 0.3613 - val_loss: 31.7182 - val_accuracy: 0.4131 - lr: 0.0100
Epoch 7/75
781/781 [=========================